In [1]:
import sympy
import numpy
import scipy
import matplotlib.pyplot as plt
import random

import datetime

import math

import pandas

import control

import scipy.optimize
from tbcontrol.responses import fopdt, sopdt
from ipywidgets import interact

import tclab

In [2]:
TCLab = tclab.setup(connected= True) #False, speedup=100)

Specifications

In [3]:
spans_deltas = 16
init_spans = 2

spans = spans_deltas+init_spans

base_unit = 60*20 #Each span lasts base_unit seconds

q_init = 50
q_final = 50

setpoint = 30

In [4]:
increment = (q_final-q_init)/(spans-1)

run_time = base_unit*spans

In [5]:
str(datetime.timedelta(seconds=run_time))

'6:00:00'

In [ ]:
with TCLab() as lab:
    h = tclab.Historian(lab.sources)
    tcplot = tclab.Plotter(h, run_time)
    q_test = q_init
    
    base_unit_checker = init_spans
    
    for t in tclab.clock(run_time):
        
        base_unit_counter = t/base_unit
        
        print(base_unit_counter, base_unit_checker, t, increment, q_test)
        
        if base_unit_counter == base_unit_checker:
            
            base_unit_checker = base_unit_counter+1
            
            increment = random.randint(40 - q_test,60 - q_test)
            while increment == 0:
                increment = random.randint(40 - q_test,60 - q_test)

            q_test += increment

        lab.Q1(q_test)
        tcplot.update(t)

In [ ]:
h.get_sessions()

In [ ]:
h.load_session(1)

In [ ]:
data = {"Time" : h.t, "T1" : h.logdict['T1'], "T2" : h.logdict['T2'], "Q1" : h.logdict['Q1'], "Q2" : h.logdict['Q2']}

In [ ]:
data_frame = pandas.DataFrame(data = data)
data_frame.to_csv("long_span_run 20200227 overnight.csv")

In [ ]:
def fopdt_pos(t, K, tau, theta, y0):
    return fopdt(t, K, numpy.exp(tau), theta, y0)

In [ ]:
for span in range(init_spans ,spans):
    lower = span*base_unit +1
    upper = (span+1)*base_unit
    
    print (data_frame['Q1'][upper-1], data_frame['Q1'][lower-2])
    
    q_change = data_frame['Q1'][upper-1] - data_frame['Q1'][lower-2]
        
    [K, tau, theta, y0], _ = scipy.optimize.curve_fit(fopdt_pos, numpy.array(h.t[lower:upper])-h.t[lower], h.logdict['T1'][lower:upper], [2, 4, 20, 50])
    tau = numpy.exp(tau)
    
    
    print([K/q_change, tau, theta, y0], q_change, lower, upper)
    tau = numpy.exp(tau)
    
#     plt.plot(h.t[lower:upper], h.logdict['T1'][lower:upper], 'go')
    plt.plot(h.t[lower:upper], fopdt(numpy.array(h.t[lower:upper])-h.t[lower], K, tau, theta, y0), 'r')